In [ ]:
!pip install vaderSentiment
!pip install nltk
!pip install stanza

In [ ]:
import requests
import re
from tqdm import tqdm
import os
from os.path import basename
import string
import operator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from os.path import join

import nltk
import nltk
nltk.download('vader_lexicon', quiet=False)

nltk.download('stopwords')

from nltk import word_tokenize,sent_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

stopwords = stopwords.words('english')

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
ana = SentimentIntensityAnalyzer()


import warnings
warnings.filterwarnings("ignore")


import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('tagsets')
nltk.download('wordnet')
nltk.download('sentiwordnet')

def remove_punctuation(words):
    new_list= []
    for w in words:
        if w.isalnum():
            new_list.append( w )
    return new_list

def ptb_to_wordnet(PTT):

    if PTT.startswith('J'):
        ## Adjective
        return 'a'
    elif PTT.startswith('V'):
        ## Verb
        return 'v'
    elif PTT.startswith('N') and not PTT.startswith('NNP'):
        ## Noune
        return 'n'
    elif PTT.startswith('R'):
        ## Adverb
        return 'r'
    else:
        return ''

def wordnet_hypernyms(token):
    all_hypernyms = []

    word_senses = wn.synsets(token)

    hypernyms = lambda s: s.hypernyms()

    for ws in word_senses:

        hypernyms = [hyp.name() for hyp in list(ws.closure(hypernyms))]
        for h in hypernyms:
            all_hypernyms.append(h[0:h.index('.')])

    return all_hypernyms

def intersection(list1,list2):
    return list(set(list1) & set(list2))


def collocation( text , regex , distance ):

    freq_c = dict()

    sentences = sent_tokenize( text )

    for sentence in sentences:

        words = word_tokenize( sentence )
        words = remove_punctuation(words)

        for i,w in enumerate(words):
            if re.search( regex , w , re.IGNORECASE ):
                index_regex = i

                for x in range( i - distance , i + distance ):
                    if x >= 0 and x < len(words) and words[x].lower() != words[index_regex].lower():
                        if len(words[x]) > 0:
                            word = words[x].lower()
                            freq_c[ word ] = freq_c.get( word , 0 ) + 1

    return freq_c


def download(directory,url):
    response = requests.get(url)
    if response:
        file_name = basename(url)
        out = open(join(directory,file_name),'w',encoding='utf-8')
        out.write(response.text)
        out.close()
    else:
      print('File cannot be downloaded')



In [ ]:
book_ids = {'29044': 'The Secret History',
 '30319086': 'If We Were Villains',
 '43263680': 'Ninth House',
 '50520939': 'The Atlas Six',
 '5297': 'The Picture Of Dorian Gray',
 '40874032': 'Vicious',
 '50548197': 'A Deadly Education',
 '50999821': 'A Lesson In Vengeance',
 '50623864': 'The Invisible Life Of Addie Larue',
 '11250317': 'The Song Of Achilles',
 '42815544': 'Bunny',
 '38633526': 'Vita Nostra',
 '23752941': 'Jane Eyre',
 '6185': 'Wuthering Heights',
 '17675462': 'The Raven Boys',
 '32571395': 'One Of Us Is Lying',
 '45300567': 'The Maidens',
 '2387172': 'Maurice',
 '22055262': 'A Darker Shade Of Magic',
 '29589074': 'Truly Devious',
 '35031085': 'Frankenstein',
 '55572506': 'A Discovery Of Witches',
 '50892240': 'They Never Learn',
 '211154709': 'King Street Run',
 '17333223': 'The Goldfinch',
 '485894': 'The Metamorphosis',
 '120274': 'The Lake Of Dead Languages',
 '23395105': 'Black Chalk',
 '6699637': 'The Magicians',
 '6514': 'The Bell Jar',
 '200704039': 'Society Of Lies',
 '63277381': 'The Fortune Seller',
 '57945316': 'Babel',
 '5941114': 'The Likeness',
 '43263485': 'The Furies',
 '61288950': 'The Cloisters',
 '34594037': 'Magic For Liars',
 '54468663': 'The Box In The Woods',
 '35154365': 'S T A G S',
 '38963039': 'People Like Us',
 '56239414': 'Clara And The Devil',
 '12224817': 'The Year Of The Gadfly',
 '44552089': 'A Lesson In Thorns',
 '53290204': 'Summer Sons',
 '41217434': 'Killing November',
 '16319': 'The Body In The Library',
 '52450064': 'The Betrayals',
 '6656': 'The Divine Comedy',
 '1556093': 'Ghost Stories Of An Antiquary',
 '41054085': 'Drawer 7',
 '7713486': 'Skippy Dies',
 '60881270': 'Laertes',
 '126917855': 'When We Were Silent',
 '197220654': 'The Book Of Dark Academia',
 '219789574': 'The Sunken Town',
 '240901431': 'The Picture Of Dorian Gray Typos Free Text Clean Copy For A Flowless Re',
 '82694054': 'Gothic Revival',
 '46259991': 'All Girls',
 '16244761': 'Extinction Point',
 '56969395': 'In The Ballroom With The Candlestick',
 '59787427': 'Over My Dead Body',
 '650775': 'Collected Ghost Stories',
 '138134': 'The Complete Poems',
 '54926634': 'Shelter For The Damned',
 '44279110': 'My Year Of Rest And Relaxation',
 '11623': 'The Unabridged Journals Of Sylvia Plath',
 '58470435': 'Psycho',
 '88340': 'Letters To Milena',
 '58897310': 'The Devil Makes Three',
 '11378763': 'The Book Of Blood And Shadow',
 '35402204': 'The Mystery Of Black Hollow Lane',
 '36467791': 'L Altra Grace',
 '61484909': 'Where Sleeping Girls Lie',
 '127279986': 'Hell Bent',
 '49798488': 'Hunting November',
 '58416952': 'The Will Of The Many',
 '7534093': 'The Lessons',
 '52439531': 'The Inheritance Games',
 '39964740': 'The Binding',
 '49535011': 'Madam',
 '45184284': 'What We Devour',
 '42379216': 'The Truants',
 '50398': 'Northanger Abbey',
 '12346651': 'The Picture Of Dorian Grey',
 '67238': 'Dead Poets Society',
 '75302266': 'A Study In Drowning',
 '101141871': 'Curious Tides',
 '6101718': 'The Magicians',
 '8667848': 'A Discovery Of Witches',
 '26856502': 'Vengeful',
 '59345249': 'The It Girl',
 '62679690': 'The Night It Ended',
 '215366272': 'Unhallowed Halls',
 '219513031': 'Academy Of Villains',
 '78816533': 'All That Consumes Us',
 '15108': 'Gentlemen And Players',
 '223138225': 'Lessons In Loyalty   Aphrodite Adonis',
 '61884783': 'Boys In The Valley',
 '234344800': 'Lessons In Trickery   Hekate Hermes',
 '61331501': 'Assassin Of Reality',
 '50485649': 'In My Dreams I Hold A Knife',
 '50892212': 'These Violent Delights',
 '43575115': 'The Starless Sea',
 '42036538': 'Gideon The Ninth',
 '50202953': 'Piranesi',
 '49203397': 'These Violent Delights',
 '9460487': 'Miss Peregrine S Home For Peculiar Children',
 '30236962': 'The Historian',
 '6334': 'Never Let Me Go',
 '42603984': 'Ace Of Spades',
 '51934838': 'Catherine House',
 '166997': 'Stoner',
 '48037': 'Carmilla',
 '62022': 'The Queen S Gambit',
 '41219': 'Possession',
 '56383038': 'Never Saw Me Coming',
 '124948406': 'They Never Learn',
 '50496875': 'Plain Bad Heroines',
 '3103': 'Maurice',
 '57800389': 'Mexican Gothic',
 '11297': 'Norwegian Wood',
 '15102': 'Gentlemen And Players',
 '42505366': 'Wilder Girls',
 '60652997': 'Hell Bent',
 '50892338': 'Legendborn',
 '57912066': 'The Atlas Paradox',
 '210223811': 'Katabasis',
 '29939230': 'A Conjuring Of Light',
 '208430658': 'Blood Over Bright Haven',
 '48896122': 'The Ivies',
 '35356380': 'People Like Us',
 '52778487': 'How We Fall Apart',
 '131177': 'Hangsaman',
 '35403058': 'City Of Ghosts',
 '207567816': 'The Scholar And The Last Faerie Door',
 '54578759': 'Have You Seen Me',
 '20764879': 'A Gathering Of Shadows',
 '200982373': 'Don T Let The Forest In',
 '21874813': 'As Chimney Sweepers Come To Dust',
 '102885': 'Waking The Moon',
 '1629601': 'The Disreputable History Of Frankie Landau Banks',
 '32895291': 'The Lying Game',
 '61054804': 'The Cloisters',
 '42614700': 'Campusland',
 '57917042': 'Anatomy',
 '10997': 'The Basic Eight',
 '1090888': 'Gaudy Night',
 '52379865': 'For Your Own Good',
 '13069261': 'The Bellwether Revivals',
 '64414866': 'An Education In Malice',
 '52179715': 'Ghosts Of Harvard',
 '55858638': 'The Golden Enclaves',
 '27068793': 'Ink And Bone',
 '44084762': 'Our Violent Ends',
 '46159063': 'The Betrayals',
 '634771': 'A Great And Terrible Beauty',
 '18431': 'The Rule Of Four',
 '229003381': 'Nightshade',
 '209563735': 'Death In The Spires',
 '16143347': 'We Were Liars',
 '40033006': 'The 7 Deaths Of Evelyn Hardcastle',
 '43232967': 'Where Sleeping Girls Lie',
 '55559887': 'The Last Graduate',
 '43744294': 'The Swallows',
 '791345': 'Picnic At Hanging Rock',
 '57426932': 'Gothikana',
 '210365385': 'The Resurrectionist',
 '35965482': 'Middlegame',
 '53063341': 'The Inheritance Games',
 '48915818': 'The Death Of Jane Lawrence',
 '174146852': 'A Dark And Drowning Tide',
 '63874788': 'In These Hallowed Halls',
 '18490': 'Frankenstein',
 '8852': 'Macbeth',
 '1381': 'The Odyssey',
 '10210': 'Jane Eyre',
 '1549': 'Antigone Oedipus The King Electra',
 '7144': 'Crime And Punishment',
 '1371': 'The Iliad',
 '112204': 'The Complete Poems Of Emily Dickinson',
 '18135': 'Romeo And Juliet',
 '17245': 'Dracula',
 '209828': 'Sappho',
 '386270': 'Selected Poems',
 '1473': 'Medea And Other Plays',
 '1715': 'Metamorphoses',
 '30289': 'The Republic',
 '81779': 'The Symposium',
 '2175': 'Madame Bovary',
 '1420': 'Hamlet',
 '223394': 'The Complete Poems',
 '119239': 'Selected Poems',
 '18386': 'The Death Of Ivan Ilych',
 '15997': 'Paradise Lost',
 '12938': 'King Lear',
 '499709': 'The Hidden Words Of Bah U Ll H',
 '17717': 'Labyrinths',
 '18495': 'The Monsters',
 '52357': 'Beowulf',
 '4934': 'The Brothers Karamazov',
 '21527802': 'Pale Fire',
 '199009': 'The Oxford Book Of English Verse',
 '12914': 'The Aeneid',
 '14942': 'Mrs Dalloway',
 '92308': 'The Importance Of Being Earnest And Other Plays',
 '13270': 'Poetics',
 '9438': 'The Basic Works Of Aristotle',
 '142247': 'Russian Short Stories From Pushkin To Buida',
 '1519': 'The Oresteia',
 '2368849': 'The Ante Room',
 '5678929': 'Bacchae And Other Plays',
 '36006882': 'De Profundis The Ballad Of Reading Gaol Other Writings',
 '28101721': 'The Greeks',
 '26252': 'The Enneads',
 '67376': 'The Book Of Love',
 '103159': 'The Forsyte Saga',
 '31491': 'The Rainbow',
 '128453': 'Complete Works',
 '169524': 'Twenty Thousand Streets Under The Sky',
 '1720478': 'Cie Wiatru',
 '1220032': 'The Rector S Daughter',
 '782982': 'The Lonely Passion Of Judith Hearne',
 '53022': 'The Collected Poems Of W B Yeats',
 '732562': 'The Rime Of The Ancient Mariner',
 '609155': 'Leaves Of Grass And Other Writings',
 '37901616': 'The Golden Treasury',
 '214229': 'Lives Of The Poets',
 '23291': 'Dubliners',
 '2090229': 'The Works Of Elizabeth Barrett Browning',
 '59716': 'To The Lighthouse',
 '32874746': 'If We Were Villains',
 '53285047': 'Bunny',
 '1914973': 'The Likeness',
 '36476218': 'She Was The Quiet One',
 '5148': 'A Separate Peace',
 '134915247': 'Babel',
 '60058055': 'The Golden Enclaves',
 '53181134': 'Mary Everything',
 '41817363': 'Tell Me Everything',
 '87365806': 'A Darker Mischief',
 '198983055': 'Curious Tides',
 '161755665': 'The Favourites',
 '238142783': 'The Quiet Girls',
 '46259689': 'Oligarchy',
 '61885117': 'This Is The Way The World Ends',
 '221142749': 'Stranger Skies',
 '198209301': 'Secrets Of The Catalogue',
 '136470356': 'Nocticadia',
 '36381099': 'All These Beautiful Strangers',
 '210446552': 'Foundation',
 '222376804': 'Oxford Blood',
 '214416570': 'The Otherwhere Post',
 '215151274': 'Love Immortal'}

In [ ]:
download('','https://raw.githubusercontent.com/peterverhaar/dark_academia/refs/heads/main/review_files.txt')


In [ ]:
txt_files = []
review_files = open('review_files.txt',encoding='utf-8')
for file in review_files:
  if re.search('txt$',file):
    txt_files.append(file.strip())



In [ ]:
base_url = 'https://raw.githubusercontent.com/peterverhaar/dark_academia/refs/heads/main/Reviews/'

txt_dir = 'Reviews'
if not os.path.isdir(txt_dir):
    os.mkdir(txt_dir)

for txt in txt_files:
  review_url = f'{base_url}{txt}'
  download(txt_dir,review_url)

In [ ]:
dir = 'Lexicon'
if not os.path.isdir(dir):
    os.mkdir(dir)

base_url = 'https://raw.githubusercontent.com/peterverhaar/dark_academia/refs/heads/main/Lexicon/'

lexicon_files = [
    'academia.txt',
    'literature_and_culture.txt',
    'mood.txt',
    'objects.txt',
    'spaces.txt'
]


for l in lexicon_files:
    topic = l[ : l.rindex('.') ]
    response = requests.get( base_url + l)
    words = []
    if response:
        response.encoding = 'utf-8'
        out = open( os.path.join( dir , l ) , 'w' , encoding = 'utf-8' )
        out.write( response.text )
        out.close()

print('Lexicons have been downloaded!')



lexicons = dict()


for file in os.listdir(dir):
    if re.search(r'txt$',file):

        topic = re.sub( r'\.txt$','',file )
        words = []

        with open( join(dir,file) , encoding = 'utf-8' ) as file_handler:
            for l in file_handler:
                if re.search( r'\w' , l ):
                    words.append(l.strip().lower())

        lexicons[topic] = words


In [ ]:
csv_directory = 'CSV'
if not os.path.exists(csv_directory):
    os.makedirs(csv_directory)


for txt_file in txt_files:

  csv = open( join( csv_directory , f'{ re.sub('[.]txt$','',txt_file) }.csv' ) , 'w' , encoding = 'utf-8' )
  csv.write('category,count\n')

  path = join(txt_dir,txt_file)

  with open(path,encoding='utf-8') as fh:

      full_text = fh.read()
      words = word_tokenize(full_text)
      words = remove_punctuation(words)
      freq = Counter(words)
      tokens = len(words)

      if tokens > 0:

          for l in lexicons:
              csv.write(f'{l},')

              count_occurrences = 0
              for word in l:
                  count_occurrences += freq.get(word.lower(),0)
              csv.write( f'{ count_occurrences / tokens}\n' )

          csv.write('\n')

  csv.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


graph_directory = 'Graphs'
if not os.path.exists(graph_directory):
    os.makedirs(graph_directory)

for csv in os.listdir(csv_directory):

  try:

    title = re.sub('[.]csv$','',csv)
    title = title[ len('review_')+1:]
    title = book_ids[title]
    print(title)

    df = pd.read_csv(join(csv_directory,csv))

    fig = plt.figure( figsize=( 7 ,6 ) )
    ax = plt.axes()

    x = 'category'
    y = 'count'

    bar_width = 0.45
    opacity = 0.8

    ax.bar( df[x] , df[y] , width = bar_width, alpha = opacity , color = '#fcc11c')

    plt.xticks(rotation= 75)

    ax.set_xlabel('Category' , fontsize= 12)
    ax.set_ylabel('Relative frequency' , fontsize = 12 )
    ax.set_title( title.title() , fontsize=20 )

    plt.ylim(0, 0.15)
    path = join( graph_directory , f'{re.sub(r'\s+','_',title.lower())}.png')
    print(path)
    plt.savefig( path ,dpi=300)
  except:
    print(f'Problem with {csv}')

In [ ]:
from google.colab import files

for file in os.listdir(graph_directory):
  files.download(join(graph_directory,file))
